In [23]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import json
from splinter import Browser
import pandas as pd
import re

In [24]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [25]:
# Define database and collection
db = client.mars_news_db
collection = db.items

In [27]:
# Scrape Mars news item
# using api for mars news website
url = "https://mars.nasa.gov/api/v1/news_items/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
response = requests.get(url)
news_items = json.loads(response.text)
news_title = news_items.get('items')[0].get('title')
news_paragraph_text = news_items.get('items')[0].get('description')
print(news_title, news_paragraph_text)

What's Mars Solar Conjunction, and Why Does It Matter? NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.


In [28]:
# Scrape Featured image
# set up splinter for mars images
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [29]:
# Get URL for image
url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url2)
html = browser.html
soup = BeautifulSoup(html, 'lxml')
temp_url = soup.find('footer').a.attrs.get('data-fancybox-href')
image_url = "https://www.jpl.nasa.gov"+temp_url
image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18182_ip.jpg'

In [30]:
# get mars weather from twitter
jsn = requests.get("https://publish.twitter.com/oembed?url=https://twitter.com/MarsWxReport/status/1163403052969336832").json()
html = jsn['html']
soup = BeautifulSoup(html,"lxml")
mars_weather = soup.find("p").get_text()
mars_weather = re.sub("pic.twitter.+","",mars_weather)
print (mars_weather)

InSight sol 258 (2019-08-18) low -100.0ºC (-148.1ºF) high -26.2ºC (-15.2ºF)winds from the SSE at 5.3 m/s (11.9 mph) gusting to 16.8 m/s (37.6 mph)pressure at 7.60 hPa 


In [73]:
# get table of mars facts from website
marsfacts_url = "https://space-facts.com/mars/"
mars_table_list = pd.read_html(marsfacts_url)
len(mars_table_list)

2

In [74]:
mars_table_list

[  Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [75]:
marsfacts_df = mars_table_list[1]
marsfacts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [76]:
marsfacts_df.columns = ['fact', 'value']

In [77]:
marsfacts_df

,fact,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [81]:
marsfacts_df.to_html(index=False, header=False).replace("\n", "")

'<table border="1" class="dataframe">  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [17]:
# Mars hemispheres images
hemisphere_image_urls = []
hemi_list = ['cerberus_enhanced', 'schiaparelli_enhanced', 'syrtis_major_enhanced', 'valles_marineris_enhanced']

for hemi in hemi_list:    
    hemi_url = []
    title = hemi.replace("_", " ")
    mars_hemi_url = f"https://astrogeology.usgs.gov/search/map/Mars/Viking/{hemi}"
    request = requests.get(mars_hemi_url)
    soup = BeautifulSoup(request.text, 'lxml')
    hemi_image_list = []
    soupy_list = soup.find_all('a', href=True)   
    
    
    for a in soupy_list:
        if f'{hemi}.tif/full' in str(a):# and 'full' not in str(a):         
            if str(a) not in hemi_image_list:
                hemi_image_list.append(str(a))
                hemi_url = hemi_image_list[0].split("\"")
                hemisphere_image_urls.append({'title': f'{title}', 'url': hemi_url[1]})
                

hemisphere_image_urls

[{'title': 'cerberus enhanced',
  'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'schiaparelli enhanced',
  'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'syrtis major enhanced',
  'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'valles marineris enhanced',
  'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]